In [1]:
import sys
print(sys.executable)
#should be AI project


C:\Users\Johanna\anaconda3\envs\AI_project\python.exe


In [19]:
!pip install xgboost

   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   - -------------------------------------- 2.6/56.8 MB 12.5 MB/s eta 0:00:05
   --- ------------------------------------ 5.2/56.8 MB 12.7 MB/s eta 0:00:05
   ----- ---------------------------------- 8.1/56.8 MB 13.2 MB/s eta 0:00:04
   -------- ------------------------------- 11.5/56.8 MB 13.9 MB/s eta 0:00:04
   ---------- ----------------------------- 15.2/56.8 MB 14.5 MB/s eta 0:00:03
   ------------- -------------------------- 18.6/56.8 MB 14.9 MB/s eta 0:00:03
   --------------- ------------------------ 21.5/56.8 MB 14.8 MB/s eta 0:00:03
   ----------------- ---------------------- 24.9/56.8 MB 15.0 MB/s eta 0:00:03
   -------------------- ------------------- 28.6/56.8 MB 15.2 MB/s eta 0:00:02
   ---------------------- ----------------- 32.2/56.8 MB 15.4 MB/s eta 0:00:02
   ------------------------- -------------- 35.7/56.8 MB 15.5 MB/s eta 0:

In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from functools import reduce
from datetime import time
from xgboost import XGBRegressor


In [3]:
df_2_full = pd.read_parquet("data/processed/df_2_full_v2.parquet")
df_2_full.head()

##otherwise
#df_2_full = pd.read_csv("data/processed/df_2_full.csv",sep=";", parse_dates=['Datetime'])

#df_2_full['Time_only'] = df_2_full['Datetime'].dt.time
#df_2_full['Date_only']=df_2_full['Datetime'].dt.date##
#df_2_full.head()


,Datetime,DP_ID,PORTAL,Interval_1,Interval_5,Interval_15,Interval_30,Interval_60,SPEED_MS_AVG,FLOW,Time_only,Date_only,PORTAL_clean,SPEED_MS_AVG_filled,FLOW_filled
0,2021-06-01 04:00:00,539,"E4S 56,160",240,48,16,8,4,19.64,9.0,04:00:00,2021-06-01,56160,19.64,9.0
1,2021-06-01 04:00:00,536,"E4S 56,160",240,48,16,8,4,28.35,2.0,04:00:00,2021-06-01,56160,28.35,2.0
2,2021-06-01 04:00:00,751,"E4S 55,620",240,48,16,8,4,21.53,19.0,04:00:00,2021-06-01,55620,21.53,19.0
3,2021-06-01 04:00:00,740,"E4S 56,160",240,48,16,8,4,21.99,17.0,04:00:00,2021-06-01,56160,21.99,17.0
4,2021-06-01 04:00:00,1254,"E4S 55,620",240,48,16,8,4,24.53,4.0,04:00:00,2021-06-01,55620,24.53,4.0


In [4]:
target_sensor = 1076
same_portal="55620"
neighbour_portal = "56160"

same_portal_sensors = df_2_full[df_2_full['PORTAL_clean'] == same_portal]['DP_ID'].unique()
same_sensors = [s for s in same_portal_sensors if s != target_sensor]


neighbour_sensors = df_2_full[df_2_full['PORTAL_clean'] == neighbour_portal]['DP_ID'].unique()
all_sensors=df_2_full['DP_ID'].unique()
except_target_sensors = [s for s in all_sensors if s != target_sensor]




prediction same time

In [5]:
df_all = []
for sensor in all_sensors:
    df_sensor = df_2_full[df_2_full['DP_ID'] == sensor].copy()
    df_sensor = df_sensor.rename(columns={
        f'FLOW_filled': f'SENSOR_{sensor}_FLOW'
    })
    df_sensor = df_sensor.rename(columns={
        f'SPEED_MS_AVG_filled': f'SENSOR_{sensor}_SPEED' 
    })
    df_sensor = df_sensor[['Datetime'] + [f'SENSOR_{sensor}_FLOW' ]+[f'SENSOR_{sensor}_SPEED' ]]
    df_all.append(df_sensor)
df_all = reduce(lambda l, r: pd.merge(l, r, on='Datetime', how='inner'), df_all)
df_all.head()
print(df_all.columns)

Index(['Datetime', 'SENSOR_539_FLOW', 'SENSOR_539_SPEED', 'SENSOR_536_FLOW',
       'SENSOR_536_SPEED', 'SENSOR_751_FLOW', 'SENSOR_751_SPEED',
       'SENSOR_740_FLOW', 'SENSOR_740_SPEED', 'SENSOR_1254_FLOW',
       'SENSOR_1254_SPEED', 'SENSOR_1076_FLOW', 'SENSOR_1076_SPEED'],
      dtype='object')


In [6]:
flow_features_same = [
    f'SENSOR_{sensor}_FLOW'
    for sensor in same_sensors
    for i in range(15)
]
flow_features_neighbour = [
    f'SENSOR_{sensor}_FLOW'
    for sensor in neighbour_sensors
    for i in range(15)
]
speed_features_same = [
    f'SENSOR_{sensor}_SPEED'
    for sensor in same_sensors
    for i in range(15)
]
speed_features_neighbour = [
    f'SENSOR_{sensor}_SPEED'
    for sensor in neighbour_sensors
    for i in range(15)
]

In [7]:

df_train, df_test = train_test_split(df_all, test_size=0.2, shuffle=False)

model_flow = LinearRegression()
model_flow.fit(df_train[flow_features_same], df_train[f'SENSOR_{target_sensor}_FLOW' ])

model_speed = LinearRegression()
model_speed.fit(df_train[speed_features_same], df_train[f'SENSOR_{target_sensor}_SPEED' ])

# prediction
y_pred = model_flow.predict(df_test[flow_features_same])
y_test=df_test[f'SENSOR_{target_sensor}_FLOW']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_pred = model_speed.predict(df_test[speed_features_same])
y_test=df_test[f'SENSOR_{target_sensor}_SPEED']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")



Baseline Linear Regression (FLOW) -> RMSE: 3.904, MAE: 3.101, R2: 0.653
Baseline Linear Regression (SPEED) -> RMSE: 1.208, MAE: 0.858, R2: 0.657


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [8]:

df_train, df_test = train_test_split(df_all, test_size=0.2, shuffle=False)

model_flow = LinearRegression()
model_flow.fit(df_train[flow_features_neighbour], df_train[f'SENSOR_{target_sensor}_FLOW' ])

model_speed = LinearRegression()
model_speed.fit(df_train[speed_features_neighbour], df_train[f'SENSOR_{target_sensor}_SPEED' ])

# prediction
y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test=df_test[f'SENSOR_{target_sensor}_FLOW']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test=df_test[f'SENSOR_{target_sensor}_SPEED']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")



Baseline Linear Regression (FLOW) -> RMSE: 3.136, MAE: 2.439, R2: 0.776
Baseline Linear Regression (SPEED) -> RMSE: 1.297, MAE: 0.901, R2: 0.605


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


prediction current time based on the last 15

In [9]:
def build_lag_features(df, target_col, lags=15):
    df = df.sort_values('Datetime')
    for i in range(1, lags + 1):
        df[f'{target_col}_lag_{i}'] = df.groupby('DP_ID')[target_col].shift(i)
    return df


In [10]:
df_lagged = build_lag_features(df_2_full, 'FLOW_filled', lags=15)
df_lagged = build_lag_features(df_lagged,"SPEED_MS_AVG_filled", lags=15)
df_lagged = df_lagged[df_lagged['FLOW_filled_lag_15'].notna()]
#decay_weights = np.exp(-np.linspace(0, 2, 15))  # z. B. λ=2
#df_lagged['FLOW_decay_feature'] = sum(
#    df_lagged[f'FLOW_filled_lag_{i+1}'] * decay_weights[i]
#    for i in range(15)
#)
print(df_lagged.isna().sum())
print(df_lagged.shape)


Datetime                          0
DP_ID                             0
PORTAL                            0
Interval_1                        0
Interval_5                        0
Interval_15                       0
Interval_30                       0
Interval_60                       0
SPEED_MS_AVG                  22876
FLOW                          22876
Time_only                         0
Date_only                         0
PORTAL_clean                      0
SPEED_MS_AVG_filled               0
FLOW_filled                       0
FLOW_filled_lag_1                 0
FLOW_filled_lag_2                 0
FLOW_filled_lag_3                 0
FLOW_filled_lag_4                 0
FLOW_filled_lag_5                 0
FLOW_filled_lag_6                 0
FLOW_filled_lag_7                 0
FLOW_filled_lag_8                 0
FLOW_filled_lag_9                 0
FLOW_filled_lag_10                0
FLOW_filled_lag_11                0
FLOW_filled_lag_12                0
FLOW_filled_lag_13          

In [11]:
df_lagged_all = []
for sensor in except_target_sensors:
    df_sensor = df_lagged[df_lagged['DP_ID'] == sensor].copy()
    df_sensor = df_sensor.rename(columns={
        f'FLOW_filled_lag_{i+1}': f'SENSOR_{sensor}_FLOW_lag_{i+1}' for i in range(15)
    })
    df_sensor = df_sensor.rename(columns={
        f'SPEED_MS_AVG_filled_lag_{i+1}': f'SENSOR_{sensor}_SPEED_lag_{i+1}' for i in range(15)
    })
    df_sensor = df_sensor[['Datetime'] + [f'SENSOR_{sensor}_FLOW_lag_{i+1}' for i in range(15)]+[f'SENSOR_{sensor}_SPEED_lag_{i+1}' for i in range(15)]]
    df_lagged_all.append(df_sensor)
df_lagged_all = reduce(lambda l, r: pd.merge(l, r, on='Datetime', how='inner'), df_lagged_all)
df_lagged_all=df_lagged_all.merge(df_2_full[df_2_full['DP_ID']==target_sensor][["Datetime","FLOW_filled","SPEED_MS_AVG_filled"]], on=['Datetime'], how='left')
#df_lagged_all = df_lagged_all.rename(columns={f'FLOW_filled': f'SENSOR_{target_sensor}_FLOW_filled'})
df_lagged_all.head()
print(df_lagged_all.columns)

Index(['Datetime', 'SENSOR_539_FLOW_lag_1', 'SENSOR_539_FLOW_lag_2',
       'SENSOR_539_FLOW_lag_3', 'SENSOR_539_FLOW_lag_4',
       'SENSOR_539_FLOW_lag_5', 'SENSOR_539_FLOW_lag_6',
       'SENSOR_539_FLOW_lag_7', 'SENSOR_539_FLOW_lag_8',
       'SENSOR_539_FLOW_lag_9',
       ...
       'SENSOR_1254_SPEED_lag_8', 'SENSOR_1254_SPEED_lag_9',
       'SENSOR_1254_SPEED_lag_10', 'SENSOR_1254_SPEED_lag_11',
       'SENSOR_1254_SPEED_lag_12', 'SENSOR_1254_SPEED_lag_13',
       'SENSOR_1254_SPEED_lag_14', 'SENSOR_1254_SPEED_lag_15', 'FLOW_filled',
       'SPEED_MS_AVG_filled'],
      dtype='object', length=153)


In [12]:
flow_features_same = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in same_sensors
    for i in range(15)
]
flow_features_neighbour = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(15)
]
speed_features_same = [
    f'SENSOR_{sensor}_SPEED_lag_{i+1}'
    for sensor in same_sensors
    for i in range(15)
]
speed_features_neighbour = [
    f'SENSOR_{sensor}_SPEED_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(15)
]

In [13]:
#saame portal



split_time = df_2_full['Datetime'].quantile(0.8)
print(split_time)

df_train = df_lagged_all[df_lagged_all['Datetime'] <= split_time]
df_test = df_lagged_all[df_lagged_all['Datetime'] > split_time]

model_flow = LinearRegression()
model_flow.fit(df_train[flow_features_same], df_train['FLOW_filled'])

model_speed = LinearRegression()
model_speed.fit(df_train[speed_features_same], df_train['SPEED_MS_AVG_filled'])



#df_lagged['FLOW_future_mean'] = df_lagged.groupby('DP_ID')['FLOW_filled'].shift(-1).rolling(15).mean()

# Vorhersage
y_pred = model_flow.predict(df_test[flow_features_same])
y_test=df_test['FLOW_filled']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_pred = model_speed.predict(df_test[speed_features_same])
y_test=df_test['SPEED_MS_AVG_filled']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")



2021-11-19 08:48:00
Baseline Linear Regression (FLOW) -> RMSE: 3.650, MAE: 2.744, R2: 0.697
Baseline Linear Regression (SPEED) -> RMSE: 1.261, MAE: 0.919, R2: 0.626


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
#neighbour

#X = df_lagged_all[features]
#y = df_lagged_all["FLOW_filled"]

split_time = df_2_full['Datetime'].quantile(0.8)
print(split_time)

df_train = df_lagged_all[df_lagged_all['Datetime'] <= split_time]
df_test = df_lagged_all[df_lagged_all['Datetime'] > split_time]

model_flow = LinearRegression()
model_flow.fit(df_train[flow_features_neighbour], df_train['FLOW_filled'])

model_speed = LinearRegression()
model_speed.fit(df_train[speed_features_neighbour], df_train['SPEED_MS_AVG_filled'])


#df_lagged['FLOW_future_mean'] = df_lagged.groupby('DP_ID')['FLOW_filled'].shift(-1).rolling(15).mean()

# Vorhersage
y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test=df_test['FLOW_filled']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test=df_test['SPEED_MS_AVG_filled']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")



2021-11-19 08:48:00
Baseline Linear Regression (FLOW) -> RMSE: 2.964, MAE: 2.239, R2: 0.800
Baseline Linear Regression (SPEED) -> RMSE: 1.288, MAE: 0.868, R2: 0.610


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Predict total flow for the next 15 mins

In [15]:
df_2_full['FLOW_future_sum'] = (
    df_2_full.groupby('DP_ID')['FLOW_filled']
    .rolling(15, min_periods=15)
    .sum()
    .shift(-14)  # damit die Summe bei t=07:02 die Werte von 07:02–07:16 enthält
    .reset_index(level=0, drop=True)
)
df_2_full['SPEED_future_mean'] = (
    df_2_full.groupby('DP_ID')['SPEED_MS_AVG_filled']
    .rolling(15, min_periods=15)
    .mean()
    .shift(-14)  # damit die Summe bei t=07:02 die Werte von 07:02–07:16 enthält
    .reset_index(level=0, drop=True)
)


df_lagged_all=df_lagged_all.merge(df_2_full[df_2_full['DP_ID']==target_sensor][["Datetime","FLOW_future_sum","SPEED_future_mean"]], on=['Datetime'], how='left')
print(df_lagged_all.shape)
df_lagged_all = df_lagged_all[df_lagged_all['Datetime'].dt.time <= time(9, 45)]
print(df_lagged_all.shape)

#print(df_lagged_all["Datetime"]dt.time.min(), df_lagged_all["Datetime"].dt.date.max())
df_lagged_all.head(20)
print(df_lagged_all.isna().sum())


(76156, 155)
(72991, 155)
Datetime                    0
SENSOR_539_FLOW_lag_1       0
SENSOR_539_FLOW_lag_2       0
SENSOR_539_FLOW_lag_3       0
SENSOR_539_FLOW_lag_4       0
                           ..
SENSOR_1254_SPEED_lag_15    0
FLOW_filled                 0
SPEED_MS_AVG_filled         0
FLOW_future_sum             0
SPEED_future_mean           0
Length: 155, dtype: int64


In [16]:
#same portal


#X = df_lagged_all[features]
#y = df_lagged_all["FLOW_future_sum"]

split_time = df_2_full['Datetime'].quantile(0.8)
print(split_time)

df_train = df_lagged_all[df_lagged_all['Datetime'] <= split_time]
df_test = df_lagged_all[df_lagged_all['Datetime'] > split_time]

model_flow = LinearRegression()
model_flow.fit(df_train[flow_features_same], df_train['FLOW_future_sum'])

model_speed = LinearRegression()
model_speed.fit(df_train[speed_features_same], df_train['SPEED_future_mean'])

#df_lagged['FLOW_future_mean'] = df_lagged.groupby('DP_ID')['FLOW_filled'].shift(-1).rolling(15).mean()

# Vorhersage
y_pred = model_flow.predict(df_test[flow_features_same])
y_test=df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")


y_pred = model_speed.predict(df_test[speed_features_same])
y_test=df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

2021-11-19 08:48:00
Baseline Linear Regression (FLOW) -> RMSE: 33.997, MAE: 22.698, R2: 0.850
Baseline Linear Regression (SPEED) -> RMSE: 0.870, MAE: 0.507, R2: 0.741


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
#neighbour portal


#X = df_lagged_all[features]
#y = df_lagged_all["FLOW_future_sum"]

split_time = df_2_full['Datetime'].quantile(0.8)
print(split_time)

df_train = df_lagged_all[df_lagged_all['Datetime'] <= split_time]
df_test = df_lagged_all[df_lagged_all['Datetime'] > split_time]

model_flow = LinearRegression()
model_flow.fit(df_train[flow_features_neighbour], df_train['FLOW_future_sum'])

model_speed = LinearRegression()
model_speed.fit(df_train[speed_features_neighbour], df_train['SPEED_future_mean'])

#df_lagged['FLOW_future_mean'] = df_lagged.groupby('DP_ID')['FLOW_filled'].shift(-1).rolling(15).mean()

# Vorhersage
y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test=df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")


y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test=df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Baseline Linear Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

2021-11-19 08:48:00
Baseline Linear Regression (FLOW) -> RMSE: 28.135, MAE: 18.771, R2: 0.897
Baseline Linear Regression (SPEED) -> RMSE: 1.015, MAE: 0.580, R2: 0.648


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


XGB-Boost

In [21]:
model_flow = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_flow.fit(df_train[flow_features_same], df_train['FLOW_future_sum'])

model_speed = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_speed.fit(df_train[speed_features_same], df_train['SPEED_future_mean'])




y_pred = model_flow.predict(df_test[flow_features_same])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

y_pred = model_speed.predict(df_test[speed_features_same])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

XGBoost Regression (FLOW) -> RMSE: 31.103, MAE: 20.831, R²: 0.875
XGBoost Regression (SPEED) -> RMSE: 0.828, MAE: 0.466, R²: 0.766


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
from xgboost import XGBRegressor
model_flow = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_flow.fit(df_train[flow_features_neighbour], df_train['FLOW_future_sum'])

model_speed = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_speed.fit(df_train[speed_features_neighbour], df_train['SPEED_future_mean'])




y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

XGBoost Regression (FLOW) -> RMSE: 25.194, MAE: 17.609, R²: 0.918
XGBoost Regression (SPEED) -> RMSE: 0.926, MAE: 0.490, R²: 0.707


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [23]:
df_lagged_all.to_parquet("data/processed/df_lagged_all.parquet", index=False)
